In [ ]:
import os
import sqlite3
con = sqlite3.connect('database.sqlite')
cursorObj = con.cursor()

Question 1 (20 points) : Write a SQL query that lists all the players born between 1987 and 1990
inclusive, sort them from the oldest to the youngest. The output of this query should be of the form:
Player Name | Birthday

In [ ]:
cursorObj.execute("SELECT player_name as [Player Name], Date(birthday) as Birthday FROM Player WHERE Birthday >= 1987 and Birthday <= 1990 ORDER BY Birthday;")

rows = cursorObj.fetchall()
 
for row in rows:
  print(row)

Question 2 (20 points) : Write a SQL query that ranks all countries and leagues based on the total
amount of total goals scored per game in the whole dataset. Sort them by the largest to the smallest
amount of goals. Note: Read this carefully. The output of this query should be of the form:
Country | League Name |Total Goals Scored

In [ ]:
cursorObj.execute("SELECT Country.name as Country, League.name as [League Name], SUM(home_team_goal+away_team_goal)as [Total Goals Scored] FROM Country ,League, Match WHERE Country.id=match.country_id AND League.id=match.league_id GROUP BY Country,[League Name] ORDER BY [Total Goals Scored] DESC;")

rows = cursorObj.fetchall()
 
for row in rows: 
  print(row)

Question 3 (20 points) : Write a SQL query that ranks all teams by the average of all their attributes (not
the players' attributes), sort them from best to worst. The output of this query should be of the form:
Team Long Name | Average of Attributes

In [ ]:
cursorObj.execute("SELECT team_long_name as [Team Long Name], AVG(buildUpPlaySpeed+ buildUpPlayDribbling+ buildUpPlayPassing+ chanceCreationPassing+ chanceCreationCrossing+ chanceCreationShooting+ defencePressure+ defenceAggression+ defenceTeamWidth)/9 as [Average of Attributes] FROM Team, Team_Attributes WHERE Team.team_api_id=Team_Attributes.team_api_id  GROUP BY team_long_name ORDER BY [Average of Attributes] DESC;")

rows = cursorObj.fetchall()
 
for row in rows: 
  print(row)

Question 4 (20 points) : Write a SQL query that ranks all teams by the average of their players'
attributes, sort them by descending order displaying only the top 5. The output of this query should be
of the form:
Team Name| Number of Players | Player Attribute Average

In [ ]:
#Not able to find relation between Team and players
cursorObj.execute("SELECT team_long_name as [Team Name],22 as [Number of Players],avg(overall_rating+potential+crossing+finishing+heading_accuracy+short_passing+volleys+dribbling+curve+free_kick_accuracy+long_passing+ball_control+acceleration+sprint_speed+agility+reactions+balance+shot_power+jumping+stamina+strength+long_shots+aggression+interceptions+positioning+vision+penalties+marking+standing_tackle+sliding_tackle+gk_diving+gk_handling+gk_kicking+gk_positioning+gk_reflexes)/35 as [Player Attribute Average] FROM Team INNER JOIN Player_Attributes ON Team.id=Player_Attributes.id GROUP BY [Team Name] ORDER BY [Player Attribute Average] LIMIT 5;")

rows = cursorObj.fetchall()
 
for row in rows: 
  print(row)

Question 5 (40 points) : Write a SINGLE SQL query that finds the date that had the most goals scored on,
per each different season and league. The output of this query should be of the form:
Date (dd/mm/yy) | Season | League Name | Goals scored

In [ ]:
cursorObj.execute("SELECT strftime('%d/%m/%Y', date) as Date, Match.season as Season, League.name as [League Name],SUM(home_team_goal+away_team_goal) as [Goals Scored] FROM Match JOIN League ON league_id=League.id GROUP BY Date, Season, [League Name] ORDER BY [Goals Scored] DESC;")
rows = cursorObj.fetchall()
 
for row in rows: 
  print(row)

Graduate Student Task (40 points) : Write a SINGLE SQL query that finds the top 5 teams in terms of
goals scored PER league for the 2008/2009 season. The output of this query should be of the form:
Season | League | Rank | Team Name | Goals Scored

In [ ]:
#Sorted Season | League | Team Name | Goals Scored but no Rank 
cursorObj.execute("SELECT season as Season, League.name as League, team_long_name as [Team Name], SUM(goals) as [Goals Scored] FROM League,(SELECT season,team_api_id,team_long_name,league_id, home_team_goal as goals FROM Team JOIN Match ON Team.team_api_id=Match.home_team_api_id WHERE season='2008/2009' Union SELECT season, team_api_id,team_long_name, league_id, away_team_goal as goals FROM Team JOIN Match ON Team.team_api_id=Match.away_team_api_id WHERE season='2008/2009')as p WHERE League.id=p.league_id GROUP BY team_long_name, League ORDER BY League, [Goals Scored] DESC;")
rows = cursorObj.fetchall()
 
for row in rows: 
  print(row)

In [ ]:
#Best try for getting Rank and filtering TOP 5 per league
#Also tried Window function but not supported by this version of sqlite
#ERROR
cursorObj.execute("SELECT Season, [League], league_rank as Rank, [Team Name], [Goals Scored] FROM (SELECT season as Season, League.name as [League], team_long_name as [Team Name], SUM(goals) as [Goals Scored], @league_rank := IF(@current_league = [League], @league_rank + 1, 1) AS league_rank, @current_league:= [League] FROM League, (SELECT season,team_api_id,team_long_name,league_id, home_team_goal as goals FROM Team JOIN Match ON Team.team_api_id=Match.home_team_api_id WHERE season='2008/2009' Union SELECT season, team_api_id,team_long_name, league_id, away_team_goal as goals FROM Team JOIN Match ON Team.team_api_id=Match.away_team_api_id WHERE season='2008/2009')ORDER BY League, [Goals Scored] DESC) ranked WHERE Rank <= 5;")
rows = cursorObj.fetchall()
 
for row in rows: 
  print(row)